Importing Necessary Libraries

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Reading Cropped Face Images From Folder, Converting Them To Gray Scale & Resizing Them Into 64*64

In [2]:
main_folder_path = "/kaggle/input/aligned-faces/aligned_faces"
X = []
y = []

for folder in os.listdir(main_folder_path):
    category_path = os.path.join(main_folder_path, folder)
    for image_name in os.listdir(category_path):
        image_path = os.path.join(category_path, image_name)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (64, 64))
#         print(img.shape)
        
        X.append(img)
        y.append(folder)

Converting X & y into Numpy Array

In [3]:
# Convert the list of strings to a NumPy array of strings
s = set(y)
print(len(s))
y = np.array(y, dtype='str')
X = np.array(X)
print(X.shape)
print(y.shape)

49
(719, 64, 64)
(719,)


One Hot Encoding y 

In [4]:
from tensorflow.keras.utils import to_categorical
label_to_int = {label: idx for idx, label in enumerate(np.unique(y))}
y_int = np.array([label_to_int[label] for label in y])

# Assuming your labels are in the form of integers (e.g., class indices)
# You should replace this with your actual labels
y_encoded = to_categorical(y_int, num_classes=49)
# Now, you can use y_train_encoded as your target data for training

In [5]:
y_int.shape

(719,)

In [6]:
y_encoded.shape

(719, 49)

Adding One More Dimension to X and Train Test Split

In [7]:
np.random.seed(1)
tf.random.set_seed(1)
X = np.expand_dims(X, axis=3)
print('Shape of X : {}, Shape of y : {}'.format(X.shape, y.shape))
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=1)

Shape of X : (719, 64, 64, 1), Shape of y : (719,)


Building Our DNN

In [8]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(64, 64, 1)),  # Flatten the 256x256x1 input
    keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(49, activation='softmax')
])

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4096)              0         
                                                                 
 dense (Dense)               (None, 512)               2097664   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0

Training

In [10]:
datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, 
                            horizontal_flip=True)
datagen.fit(X_train)
batch_size=32
model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch =
                          len(X_train) // batch_size, epochs=200, validation_data=(X_test, y_test))

Epoch 1/200


/tmp/ipykernel_29/3313357855.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch =


15/15 [==============================] - 5s 25ms/step - loss: 20.4543 - accuracy: 0.0977 - val_loss: 331.2533 - val_accuracy: 0.0509
Epoch 2/200
15/15 [==============================] - 0s 9ms/step - loss: 17.0513 - accuracy: 0.2675 - val_loss: 483.5902 - val_accuracy: 0.0463
Epoch 3/200
15/15 [==============================] - 0s 9ms/step - loss: 13.9282 - accuracy: 0.4586 - val_loss: 864.5911 - val_accuracy: 0.0278
Epoch 4/200
15/15 [==============================] - 0s 8ms/step - loss: 11.5807 - accuracy: 0.5520 - val_loss: 918.1453 - val_accuracy: 0.0231
Epoch 5/200
15/15 [==============================] - 0s 8ms/step - loss: 9.8608 - accuracy: 0.6497 - val_loss: 884.2158 - val_accuracy: 0.0324
Epoch 6/200
15/15 [==============================] - 0s 9ms/step - loss: 8.6055 - accuracy: 0.6879 - val_loss: 807.4407 - val_accuracy: 0.0833
Epoch 7/200
15/15 [==============================] - 0s 13ms/step - loss: 7.5835 - accuracy: 0.7417 - val_loss: 1019.2602 - val_accuracy: 0.0417
Epoc

Normalizing Test Data & Predicting

In [11]:
from keras.backend import epsilon
def normalize(X, mean, std):
    return (X - mean) / (std + epsilon())

predicted = model.predict(normalize(X_test, datagen.mean, datagen.std))

7/7 [==============================] - 0s 2ms/step


In [12]:
predicted.shape

(216, 49)

In [13]:
y_test.shape

(216, 49)

Accuracy On Unseen Test Data

In [14]:
import numpy as np

# Assuming 'y_pred' is your predicted softmax distribution and 'y_test' is your true labels

# Convert softmax predictions to class indices (argmax)
y_pred_classes = np.argmax(predicted, axis=1)

# Convert one-hot encoded true labels to class indices (argmax)
y_true_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = np.mean(y_pred_classes == y_true_classes)
print(accuracy)

0.9398148148148148
